In [ ]:
!pip install tqdm scidownl\

In [7]:
from scidownl import scihub_download
import pandas as pd
from tqdm import tqdm
import concurrent.futures

In [1]:
def download_papers(paper):
    name = paper.replace('/', ':')
    scihub_download(paper, paper_type="doi", out=f"./papers/{name}.pdf")

df = pd.read_csv('output.csv')
source = list(df['DOI'])
source = [doi for doi in source if pd.notna(doi)]

In [3]:
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Submit tasks to the executor
    futures = [executor.submit(download_papers, paper) for paper in source]

    # Iterate over completed futures to check progress
    for _ in tqdm(concurrent.futures.as_completed(futures), total=len(source)):
        pass

  0%|          | 0/301 [00:00<?, ?it/s][INFO] | 2023/07/10 13:41:42 | Choose scihub url [0]: https://sci-hub.se
[INFO] | 2023/07/10 13:41:42 | <- Request: scihub_url=https://sci-hub.se, source=DoiSource[type=doi, id=10.1016/j.apsoil.2023.104931], proxies={}


[INFO] | 2023/07/10 13:41:42 | -> Response: status_code=403, content_length=11541
[WARNING] | 2023/07/10 13:41:42 | Error occurs, task status: crawling_failed, error: Error occurs when crawling source: DoiSource[type=doi, id=10.1016/j.apsoil.2023.104931]
[INFO] | 2023/07/10 13:41:42 | Choose scihub url [1]: http://sci-hub.ru
[INFO] | 2023/07/10 13:41:42 | <- Request: scihub_url=http://sci-hub.ru, source=DoiSource[type=doi, id=10.1016/j.apsoil.2023.104931], proxies={}
[INFO] | 2023/07/10 13:41:43 | -> Response: status_code=403, content_length=11541
[WARNING] | 2023/07/10 13:41:43 | Error occurs, task status: crawling_failed, error: Error occurs when crawling source: DoiSource[type=doi, id=10.1016/j.apsoil.2023.104931]
[INFO] | 2023/07/10 13:41:43 | Choose scihub url [2]: http://sci-hub.st
[INFO] | 2023/07/10 13:41:43 | <- Request: scihub_url=http://sci-hub.st, source=DoiSource[type=doi, id=10.1016/j.apsoil.2023.104931], proxies={}
[INFO] | 2023/07/10 13:41:43 | -> Response: status_code=

100% [==================================================] 8236245/8236245


[INFO] | 2023/07/10 13:43:06 | ↓ Successfully download the url to: ./paper/10.3390:w13121677.pdf
 21%|██        | 62/301 [01:23<10:29,  2.63s/it][INFO] | 2023/07/10 13:43:06 | Choose scihub url [0]: https://sci-hub.se
[INFO] | 2023/07/10 13:43:06 | <- Request: scihub_url=https://sci-hub.se, source=DoiSource[type=doi, id=10.1051/e3sconf/202126402032], proxies={}
[INFO] | 2023/07/10 13:43:06 | -> Response: status_code=200, content_length=5837
[WARNING] | 2023/07/10 13:43:06 | Error occurs, task status: extracting_failed, error: No pdf tag was found in the given content with the selector: #pdf
[INFO] | 2023/07/10 13:43:06 | Choose scihub url [1]: http://sci-hub.ru
[INFO] | 2023/07/10 13:43:06 | <- Request: scihub_url=http://sci-hub.ru, source=DoiSource[type=doi, id=10.1051/e3sconf/202126402032], proxies={}
[INFO] | 2023/07/10 13:43:06 | -> Response: status_code=403, content_length=11541
[WARNING] | 2023/07/10 13:43:06 | Error occurs, task status: crawling_failed, error: Error occurs when 

100% [==================================================] 8301454/8301454


...

[INFO] | 2023/07/10 13:49:59 | ↓ Successfully download the url to: ./paper/10.1097:00010694-198301000-00010.pdf
 99%|█████████▉| 298/301 [08:17<00:04,  1.44s/it][INFO] | 2023/07/10 13:49:59 | Choose scihub url [0]: https://sci-hub.se
[INFO] | 2023/07/10 13:49:59 | <- Request: scihub_url=https://sci-hub.se, source=DoiSource[type=doi, id=10.2753/pet1061-1991200469], proxies={}
[INFO] | 2023/07/10 13:50:00 | -> Response: status_code=200, content_length=7039
[INFO] | 2023/07/10 13:50:00 | * Extracted information: {'url': 'https://dacemirror.sci-hub.se/journal-article/65d1a815b748ff763b2f1dbf3944459a/loiter1977.pdf', 'title': 'The Effectiveness Of Capital Investments In The Protection Of The Natural Environment. Problems in Economics, 20(4), 69–88'}


100% [==================================================] 1172660/1172660


[INFO] | 2023/07/10 13:50:01 | ↓ Successfully download the url to: ./paper/10.2753:pet1061-1991200469.pdf
 99%|█████████▉| 299/301 [08:19<00:03,  1.60s/it][INFO] | 2023/07/10 13:50:01 | Choose scihub url [0]: https://sci-hub.se
[INFO] | 2023/07/10 13:50:01 | <- Request: scihub_url=https://sci-hub.se, source=DoiSource[type=doi, id=10.1016/0016-7061(76)90069-0], proxies={}
[INFO] | 2023/07/10 13:50:02 | -> Response: status_code=200, content_length=6976
[INFO] | 2023/07/10 13:50:02 | * Extracted information: {'url': 'https://zero.sci-hub.se/1592/5eb4ea06ea9780d53284fb02a831a11a/borovsky1976.pdf', 'title': 'Soil processes and productivity in relation to climatic cycles in Kazakhstan. Geoderma, 15(1), 41–49'}


100% [==================================================] 527800/527800


[INFO] | 2023/07/10 13:50:02 | ↓ Successfully download the url to: ./paper/10.1016:0016-7061(76)90069-0.pdf
100%|█████████▉| 300/301 [08:19<00:01,  1.38s/it][INFO] | 2023/07/10 13:50:02 | Choose scihub url [0]: https://sci-hub.se
[INFO] | 2023/07/10 13:50:02 | <- Request: scihub_url=https://sci-hub.se, source=DoiSource[type=doi, id=10.5586/asbp.1975.052], proxies={}
[INFO] | 2023/07/10 13:50:03 | -> Response: status_code=200, content_length=5830
[WARNING] | 2023/07/10 13:50:03 | Error occurs, task status: extracting_failed, error: No pdf tag was found in the given content with the selector: #pdf
[INFO] | 2023/07/10 13:50:03 | Choose scihub url [1]: http://sci-hub.ru
[INFO] | 2023/07/10 13:50:03 | <- Request: scihub_url=http://sci-hub.ru, source=DoiSource[type=doi, id=10.5586/asbp.1975.052], proxies={}
[INFO] | 2023/07/10 13:50:03 | -> Response: status_code=403, content_length=11541
[WARNING] | 2023/07/10 13:50:03 | Error occurs, task status: crawling_failed, error: Error occurs when cr

In [9]:
import os

folder_path = "papers"  # Replace "papers" with your folder path

# Get the list of files in the folder
file_list = os.listdir(folder_path)

# Count the number of files
num_files = len(file_list)

print(f"The number of files in the folder '{folder_path}' is: {num_files} out of {len(source)} initial DOIs")

The number of files in the folder 'papers' is: 195 out of 301 initial DOIs


#### PDF Downloader

In [9]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm


In [15]:
df = pd.read_csv('output.csv')
source = list(df['DOI'])
print(len(source))

source = [doi for doi in source if pd.notna(doi)]
print(len(source))

350
301


In [19]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from itertools import cycle

titles = []
citations = []
dois = []

proxy_list = [
    'http://123.45.67.89:8080',
    'http://192.168.1.100:3128',
    'http://54.32.10.12:8888',
    'http://210.55.33.44:80',
    'http://176.99.99.99:8081',
    'http://67.205.146.164:3129',
    'http://45.76.101.39:8118',
    'http://89.123.67.45:9999',
    'http://103.14.250.6:53281',
    'http://87.98.219.129:3128',
    'http://91.206.30.218:80',
    'http://172.104.186.156:3128',
    'http://109.245.212.134:8080',
    'http://217.219.67.6:8080',
    'http://201.231.137.12:3128',
    'http://203.176.135.90:8080',
    'http://95.110.194.245:8080',
    'http://45.32.124.53:8080',
    'http://186.219.40.186:3128',
    'http://123.200.14.130:8080'
]


proxy_cycle = cycle(proxy_list)

def pdf_downloader(doi):
    doi = doi.replace(':', '/')
    base_url = "https://sci-hub.ru"
    url = f"{base_url}/{doi}"

    proxy = next(proxy_cycle)
    proxies = {
        'http': proxy,
        'https': proxy
    }

    try:
        response = requests.get(url, proxies=proxies)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')

        citation_div = soup.find('div', {'id': 'citation'})
        title = citation_div.find('i').text
        citation = citation_div.text
        titles.append(title)
        citations.append(citation)
        dois.append(doi)

        pdf_relative_url = soup.find('embed', {'type': 'application/pdf'}).get('src')
        pdf_url = urljoin(base_url, pdf_relative_url)

        pdf_response = requests.get(pdf_url, proxies=proxies)

        filename = f"data/{doi.replace('/', ':')}.pdf"

        with open(filename, "wb") as file:
            file.write(pdf_response.content)

    except Exception as e:
        titles.append(None)
        citations.append(None)
        dois.append(doi)
        print(f"An exception occurred: {str(e)}")

# Create a ThreadPoolExecutor with a maximum of 5 concurrent workers
with ThreadPoolExecutor(max_workers=5) as executor:
    # Submit tasks to the executor
    futures = [executor.submit(pdf_downloader, doi) for doi in doi_list]

    # Iterate over completed futures to check for any exceptions
    for future in tqdm(as_completed(futures), total=len(futures)):
        try:
            future.result()
        except Exception as e:
            print(f"An exception occurred: {str(e)}")


  0%|          | 1/301 [01:15<6:17:05, 75.42s/it]

An exception occurred: HTTPSConnectionPool(host='sci-hub.ru', port=443): Max retries exceeded with url: /10.1016/j.scitotenv.2023.163226 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x11f409360>, 'Connection to 54.32.10.12 timed out. (connect timeout=None)'))An exception occurred: HTTPSConnectionPool(host='sci-hub.ru', port=443): Max retries exceeded with url: /10.3390/agriculture13030704 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x11f408af0>, 'Connection to 176.99.99.99 timed out. (connect timeout=None)'))

An exception occurred: HTTPSConnectionPool(host='sci-hub.ru', port=443): Max retries exceeded with url: /10.1016/j.apsoil.2023.104931 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x11f4091e0>, 'Connection to 123.45.67.89 timed out. (connect timeout=None)'))
An exception occurred: HTTPSConnectionPool(host='sci-hub.ru', port=443): Max retries exceeded with url: /10.1016/j.rsase.2023.100

  2%|▏         | 6/301 [01:15<45:45,  9.31s/it]  

An exception occurred: HTTPSConnectionPool(host='sci-hub.ru', port=443): Max retries exceeded with url: /10.1038/d41586-023-00021-4 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x281b60130>: Failed to establish a new connection: [Errno 61] Connection refused')))
An exception occurred: HTTPSConnectionPool(host='sci-hub.ru', port=443): Max retries exceeded with url: /10.1007/978-3-031-26592-1_9 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x11f40bdf0>: Failed to establish a new connection: [Errno 61] Connection refused')))


  3%|▎         | 8/301 [01:16<30:33,  6.26s/it]

...

  4%|▍         | 12/301 [02:30<51:19, 10.65s/it] 

An exception occurred: HTTPSConnectionPool(host='sci-hub.ru', port=443): Max retries exceeded with url: /10.1007/s41748-022-00335-9 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x11f4098a0>, 'Connection to 91.206.30.218 timed out. (connect timeout=None)'))
An exception occurred: HTTPSConnectionPool(host='sci-hub.ru', port=443): Max retries exceeded with url: /10.1038/s41598-022-15802-6 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x11f4098d0>: Failed to establish a new connection: [Errno 61] Connection refused')))


  5%|▍         | 14/301 [02:30<35:50,  7.49s/it]

An exception occurred: HTTPSConnectionPool(host='sci-hub.ru', port=443): Max retries exceeded with url: /10.1029/2022WR032614 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x11f40ae30>: Failed to establish a new connection: [Errno 61] Connection refused')))An exception occurred: HTTPSConnectionPool(host='sci-hub.ru', port=443): Max retries exceeded with url: /10.1007/s42773-022-00148-z (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x11f409330>: Failed to establish a new connection: [Errno 61] Connection refused')))

An exception occurred: HTTPSConnectionPool(host='sci-hub.ru', port=443): Max retries exceeded with url: /10.1007/s11852-022-00909-9 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x11f40b400>, 'Connection to 109.245.212.134 timed out. (connect timeout=None)'))


  5%|▌         | 16/301 [02:54<51:41, 10.88s/it]


In [7]:
import requests
import pandas as pd
from lxml.html import fromstring

def to_get_proxies():
    url = 'https://free-proxy-list.net/'

    response = requests.get(url)

    parser = fromstring(response.text)
    headers = [header.text_content() for header in parser.xpath('//thead/tr/th')]
    rows = parser.xpath('//tbody/tr')

    proxies = []
    for row in rows:
        proxy = [cell.text_content() for cell in row.xpath('.//td')]
        proxies.append(proxy)

    proxies_df = pd.DataFrame(proxies, columns=headers)
    return proxies_df


In [8]:
to_get_proxies()

,IP Address,Port,Code,Country,Anonymity,Google,Https,Last Checked
0,50.173.157.75,80,US,United States,anonymous,no,no,8 secs ago
1,91.229.114.137,80,SG,Singapore,anonymous,yes,no,8 secs ago
2,50.207.199.86,80,US,United States,anonymous,no,no,8 secs ago
3,50.169.62.107,80,US,United States,anonymous,no,no,8 secs ago
4,195.23.57.78,80,PT,Portugal,anonymous,no,no,8 secs ago
...,...,...,...,...,...,...,...,...
304,2023-07-07,945,366,4%,117,536,292,6645
305,2023-07-06,740,348,5%,87,432,221,6582
306,917,392,3%,119,511,287,6450,None
307,423,180,3%,54,235,132,2976,None
